In [1]:
from src import cropy as cpy
import json
import geopandas as gpd
import numpy as np
import rioxarray
import xarray as xr
#https://nbviewer.jupyter.org/github/pangeo-data/cog-best-practices/blob/main/0-single-cog.ipynb
import os
os.environ['GDAL_DISABLE_READDIR_ON_OPEN']='EMPTY_DIR' #This is KEY! otherwise we send a bunch of HTTP GET requests to test for common sidecar metadata
os.environ['AWS_NO_SIGN_REQUEST']='YES' #Since this is a public bucket, we don't need authentication
os.environ['GDAL_MAX_RAW_BLOCK_CACHE_SIZE']='8000000000'  #800MB: Want this to be greater than size of uncompressed raster to overcome a 10 MB limit in the GeoTIFF driver for range request merging.
os.environ['GDAL_SWATH_SIZE']='8000000000'  #also increase this if increasing MAX_RAW_BLOCK_CACHE_SIZE
os.environ['VSI_CURL_CACHE_SIZE']='8000000000' #also increase this if increasing MAX_RAW_BLOCK_CACHE_SIZE8

In [2]:
'''
target_aoi={
  "type": "FeatureCollection",
  "features": [
    {
      "type": "Feature",
      "properties": {},
      "geometry": {
        "type": "Polygon",
        "coordinates": [
          [
            [
              30.069580078125,
              36.29741818650811
            ],
            [
              30.65185546875,
              36.29741818650811
            ],
            [
              30.65185546875,
              36.94989178681327
            ],
            [
              30.069580078125,
              36.94989178681327
            ],
            [
              30.069580078125,
              36.29741818650811
            ]
          ]
        ]
      }
    }
  ]
}
'''

'\ntarget_aoi={\n  "type": "FeatureCollection",\n  "features": [\n    {\n      "type": "Feature",\n      "properties": {},\n      "geometry": {\n        "type": "Polygon",\n        "coordinates": [\n          [\n            [\n              30.069580078125,\n              36.29741818650811\n            ],\n            [\n              30.65185546875,\n              36.29741818650811\n            ],\n            [\n              30.65185546875,\n              36.94989178681327\n            ],\n            [\n              30.069580078125,\n              36.94989178681327\n            ],\n            [\n              30.069580078125,\n              36.29741818650811\n            ]\n          ]\n        ]\n      }\n    }\n  ]\n}\n'

In [68]:
#datajson=json.dumps(target_aoi)
#area=gpd.read_file(datajson)


In [3]:
target_aoi=open("target_aoi/tr-south.json")
area=gpd.read_file(target_aoi)

In [4]:
cpy.VectorProcessing.show_vector(target_area=area)

In [5]:
m,intersec_df=cpy.VectorProcessing.show_intersection(target_area=area,base_vector_path='sentinel_tiles/sentinel_tr_tiles.shp')

In [6]:
m

In [12]:
boundry=list(area.geometry.bounds.values[0]) #boundry from your AOI
bbox=[boundry[0],boundry[1],boundry[2],boundry[3]] #(min lon, min lat, max lon, max lat)
dates = '2021-03-01/2021-04-30'
band_list=['B02','B08']
cloud_percentage=10

In [8]:
boundry

[34.03564453125, 36.03133177633187, 42.890625, 38.659777730712534]

In [13]:
stac_query=cpy.Stac(target_aoi=bbox,date=dates,max_cloud=cloud_percentage)

In [14]:
sentinel_items=stac_query.find_sentinel_item()

In [15]:
cpy.Stac.show_result_map(items_list=sentinel_items,
                         overview=True,
                         #target_area=target_aoi
                         target_area=area)

In [90]:
stac_query.find_time_series()

In [91]:
cpy.Stac.show_result_map(items_list=stac_query.stac_items,
                         overview=True,
                         #target_area=target_aoi
                         target_area=area)

In [94]:
len(stac_query.stac_items)

55

In [106]:
stac_query.tiles_list

['35SQA', '35SQB', '36STF', '36STG']

In [100]:
it=stac_query.stac_items[26]
it.properties

{'datetime': '2021-03-12T08:49:56Z',
 'platform': 'sentinel-2b',
 'constellation': 'sentinel-2',
 'instruments': ['msi'],
 'gsd': 10,
 'view:off_nadir': 0,
 'proj:epsg': 32636,
 'sentinel:utm_zone': 36,
 'sentinel:latitude_band': 'S',
 'sentinel:grid_square': 'TG',
 'sentinel:sequence': '0',
 'sentinel:product_id': 'S2B_MSIL2A_20210312T083719_N0214_R064_T36STG_20210312T112236',
 'sentinel:data_coverage': 74.5,
 'eo:cloud_cover': 3.73,
 'sentinel:valid_cloud_cover': True,
 'created': '2021-03-13T05:24:31.626Z',
 'updated': '2021-03-13T05:24:31.626Z'}

In [105]:
it.bbox

[29.78527471375086, 36.91436260952561, 30.86396348048385, 37.927671217101775]

In [ ]:
'S2B_OPER_MSI_L1C_TL_VGS2_20210312T105042_A020968_T36STG_N02.09'
'S2B_MSIL2A_20210312T083719_N0214_R064_T36STG_20210312T112236'

In [97]:
for i in stac_query.stac_items:
    print(i.properties['sentinel:product_id'])

S2A_MSIL2A_20210529T084601_N0300_R107_T35SQA_20210529T115308
S2A_MSIL2A_20210526T083601_N0300_R064_T35SQA_20210526T122012
S2B_MSIL2A_20210524T084559_N0300_R107_T35SQA_20210524T111238
S2B_MSIL2A_20210521T083559_N0300_R064_T35SQA_20210521T113326
S2A_MSIL2A_20210519T084601_N0300_R107_T35SQA_20210519T134205
S2A_MSIL2A_20210516T083601_N0300_R064_T35SQA_20210516T115317
S2B_MSIL2A_20210514T084559_N0300_R107_T35SQA_20210514T113538
S2B_MSIL2A_20210511T083559_N0300_R064_T35SQA_20210512T152628
S2A_MSIL2A_20210509T084601_N0300_R107_T35SQA_20210509T115513
S2A_MSIL2A_20210506T083601_N0300_R064_T35SQA_20210506T103959
S2A_MSIL2A_20210426T083601_N0300_R064_T35SQA_20210426T122131
S2B_MSIL2A_20210421T083559_N0300_R064_T35SQA_20210421T113135
S2B_MSIL2A_20210411T083559_N0300_R064_T35SQA_20210411T113126
S2A_MSIL2A_20210327T083601_N0214_R064_T35SQA_20210327T113705
S2A_MSIL2A_20210310T084801_N0214_R107_T35SQA_20210310T113842
S2A_MSIL2A_20210529T084601_N0300_R107_T36STG_20210529T115308
S2A_MSIL2A_20210526T0836

In [ ]:
cpy.Stac.show_result_list(sentinel_items)[0:2]

In [ ]:
cpy.Stac.show_result_df(sentinel_items_list=sentinel_items)

In [ ]:
band_list=['B01', 'B02', 'B03', 'B04', 'B05', 'B06', 'B07', 'B08', 'B8A',
           'B09', 'B11', 'B12', 'AOT', 'WVP', 'SCL', 'info', 'metadata',
           'visual', 'overview', 'thumbnail']

In [ ]:
%%time
#download all sentinel_items with target bands
cpy.Stac.download_image(item_list=sentinel_items,
                        band_list=band_list[0:4],                                           
                        download_path='./target_path2'
                                              )

In [ ]:
%%time
# get data as xarray or open download_status and save the data to download_path
subset_image=stac_query.download_subset_image(item_list=sentinel_items,
                                              band_list=['B02', 'B08','B03','B04'],
                                              aoi=target_aoi,
                                              download_status=True,
                                              download_path='./target_path2_subset'
                                              )

In [ ]:
%%time
# get data as xarray or open download_status and save the data to download_path
subset_image=stac_query.download_subset_image2(item_list=sentinel_items,
                                              band_list=['B02','B03','B04','B08'],
                                              aoi=target_aoi,
                                              download_status=True,
                                              download_path='./target_path2_subset',
                                              send_s3=True,
                                              bucket_name='sentinel2-doktar', 
                                              delete_local=False)

In [ ]:
%%time
#download all sentinel_items with target bands
cpy.Stac.download_image(item_list=sentinel_items,
                        band_list=band_list[0:4],
                        download_path='./target_path2'
                        )

In [ ]:
%%time
# get data as xarray and open download_status and save the data to download_path
subset_image=stac_query.download_subset_image(item_list=sentinel_items,
                                              band_list=['B02', 'B08','B03','B04'],
                                              aoi=target_aoi,
                                              download_status=True,
                                              download_path='./target_path3'
                                              )

### FIND TIME SERIES DATA

In [ ]:
from src import cropy as cpy
import json
import geopandas as gpd
import numpy as np
target_aoi={
  "type": "FeatureCollection",
  "features": [
    {
      "type": "Feature",
      "properties": {},
      "geometry": {
        "type": "Polygon",
        "coordinates": [
          [
            [
              -2.79052734375,
              42.33418438593939
            ],
            [
              -1.9335937499999998,
              42.33418438593939
            ],
            [
              -1.9335937499999998,
              43.229195113965005
            ],
            [
              -2.79052734375,
              43.229195113965005
            ],
            [
              -2.79052734375,
              42.33418438593939
            ]
          ]
        ]
      }
    }
  ]
}
datajson=json.dumps(target_aoi)
area=gpd.read_file(datajson)

#cpy.VectorProcessing.show_vector(target_area=area)
m,intersec_df=cpy.VectorProcessing.show_intersection(target_area=area,base_vector_path='sentinel_tiles/sentinel_tr_tiles.shp')
#m
boundry=list(area.geometry.bounds.values[0]) #boundry from your AOI
bbox=[boundry[0],boundry[1],boundry[2],boundry[3]] #(min lon, min lat, max lon, max lat)
dates = '2021-05-02/2021-12-10'
band_list=['B02','B08']
cloud_percentage=80


In [ ]:
stac_query=cpy.Stac(target_aoi=bbox,date=dates,max_cloud=cloud_percentage)


In [ ]:
coverage=sorted(stac_query.stac_items.properties('sentinel:data_coverage'))
#self.min_coverage=95. 95 den buyuk coverage sahip goruntuleri aliyoruz
coverage_sorted=[c for c in coverage if c>=95]
coverage_sorted

In [ ]:
if coverage_sorted:
    stac_query.stac_items.filter('sentinel:data_coverage',coverage_sorted)
else:
    coverage_sorted=coverage[-2:]
    stac_query.stac_items.filter('sentinel:data_coverage',coverage_sorted)

if 100 in stac_query.stac_items.properties('sentinel:data_coverage'):
    stac_query.stac_items.filter('sentinel:data_coverage', [100])

In [ ]:
t_list=stac_query.create_tiles_list()
t_list

In [ ]:
stac_query.tiles_list=t_list
stac_query.find_time_series()


In [ ]:
sorted(stac_query.stac_items.properties('eo:cloud_cover'))[0]

In [ ]:
#print(sorted(stac_query.stac_items.properties('eo:cloud_cover')))
print(sorted(stac_query.stac_items.properties('sentinel:data_coverage')))
#print(stac_query.stac_items.summary())

In [ ]:
sorted(stac_query.stac_items.properties('eo:cloud_cover'))

In [ ]:
#stac_query.stac_items.filter('eo:cloud_cover', [0])
stac_query.stac_items.filter('sentinel:data_coverage', [100])

In [ ]:
len(stac_query.stac_items)

In [ ]:
bb=stac_query.stac_items[1]

In [ ]:
bb.properties

In [ ]:
cpy.Stac.show_result_map(items_list=stac_query.stac_items,
                         overview=True,
                         #target_area=target_aoi
                         target_area=area)

In [ ]:
#cpy.Stac.show_result_list(stac_query.stac_items)
res_df=cpy.Stac.show_result_df(sentinel_items_list=stac_query.stac_items)

In [ ]:
res_df.head(10)

In [ ]:
cpy.Stac.show_result_map(items_list=stac_query.stac_items,
                         overview=True,
                         target_area=target_aoi)

In [ ]:
tar_items_list=list(res_df['sentinel:product_id'])[0:6]

In [ ]:
items_list=['S2B_MSIL2A_20180915T080559_N0208_R078_T37SDA_20180915T123831','S2A_MSIL2A_20180910T080601_N0208_R078_T37SDA_20180910T130532']

In [ ]:
cpy.Stac.download_image(stac_result=stac_query.stac_result,
                        item_id_list=tar_items_list,
                        band_list=['B04','B08'],
                        download_path='time_series_path')

In [ ]:
%%time
subset_2bands=cpy.Stac.download_subset_image(stac_result=stac_query.stac_result,
                                             item_id_list=tar_items_list,
                                             aoi=target_aoi,
                                             band_list=['B04','B08'],)

### Calculate Image Statistic
Below function, calculate the statistic of each band in subset image list. <br> 
It'll be more real example when you use this method with time series data example <br>
On the same Sentinel-2 tile, you can get stat (mean,median,min,max) value at your target time range

In [ ]:
%%time
st=cpy.calc_img_stat(stac_result=subset_2bands,statistic_method='median')